In [ ]:
NdFK8CWBYMTDOy1TfjiApAwKgg4LnEPA

{
    "pages": [
        {
            "index": 0,
            "markdown": "# Shai Gusto Restaurant 2011 Hewitt Ave Everett WA 98201 <br> Tel. 425-252-3525 Fax. 425-252-3525 <br> Welcome to Thai Gusto \n\n## Table 4\n\nDate: 6/16/2017 12:36 PM\nServer: PRACH\nChang (2\\$3.50) \\$7.00\n\\#L14 Garlic Delight \\$10.24\n[Chicken][5*****][Smal1 Brown Rice]\n\\#L28 Red Curry \\$10.24\n[Chicken][5*****][Smal1 Brown Rice]\nTotal 4 item(s) \\$27.48\nSales Tax \\$2.67\n$\\qquad$\n$\\qquad$\n\n## Grand Total $\\$ 30.15$\n\nTip Guide\n$15 \\%=\\$ 4.52, \\quad 18 \\%=\\$ 5.43, \\quad 20 \\%=\\$ 6.03$\n\nThank you very much.\nCome back again.\n\n- GUEST CHECK -\n\nThis is not a receipt.",
            "images": [],
            "dimensions": {
                "dpi": 200,
                "height": 1000,
                "width": 750
            }
        }
    ],
    "model": "mistral-ocr-2503-completion",
    "usage_info": {
        "pages_processed": 1,
        "doc_size_bytes": 132421
    }
}


In [41]:
image_ocr_markdown = response_dict["pages"][0]["markdown"]

In [75]:
# Initialize Mistral client with API key
from mistralai import Mistral
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.environ["MISTRAL_KEY"] # Replace with your API key
client = Mistral(api_key=api_key)

# Import required libraries
from pathlib import Path
from mistralai import DocumentURLChunk, ImageURLChunk, TextChunk
import json

# Verify PDF file exists
import base64

# Verify image exists
image_file = Path("/Users/eliot/Desktop/project_ML/dataset/receipts/1027-receipt.jpg")
assert image_file.is_file()

# Encode image as base64 for API
encoded = base64.b64encode(image_file.read_bytes()).decode()
base64_data_url = f"data:image/jpeg;base64,{encoded}"

image_response = client.ocr.process(
    document=ImageURLChunk(image_url=base64_data_url),
    model="mistral-ocr-latest"
)

# Convert response to JSON
response_dict = json.loads(image_response.model_dump_json())
json_string = json.dumps(response_dict, indent=4)

image_ocr_markdown = response_dict["pages"][0]["markdown"]
# Get structured response from model
chat_response = client.chat.complete(
    model="pixtral-12b-latest",
    messages=[
        {
            "role": "user",
            "content": [
                ImageURLChunk(image_url=base64_data_url),
                TextChunk(
                    text=(
                        f"This is image's OCR in markdown:\n\n{image_ocr_markdown}\n.\n"
                        "I wan't same structured json with only 'name' restaurant final 'total' and 'date', and the 'adress'"
                        "The output should be strictly be json with no extra commentary"
                        
                    )
                ),
            ],
        }
    ],
    response_format={"type": "json_object"},
    temperature=0,
)

# Parse and return JSON response
response_dict = json.loads(chat_response.choices[0].message.content)



In [76]:
response_dict

{'restaurant': {'name': 'Longboards',
  'address': '92-161 Waipahe Place, Kapolei, HI 96707'},
 'total': 44.31,
 'date': '01/11/2019'}

In [74]:
import pandas

In [79]:

import os
import pandas as pd

folder_path = "/Users/eliot/Desktop/project_ML/dataset/bank_statements"
all_dfs = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # You can change this for other file types
        full_path = os.path.join(folder_path, filename)
        df = pd.read_csv(full_path)
        all_dfs.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(all_dfs, ignore_index=True)

combined_df



,date,amount,currency,vendor
0,2016-09-01,7.61,USD,"7230 Pendleton Pike, Indianapolis, IN 46226"
1,2018-07-21,25.78,USD,"889 W. 190th Street, Los Angeles, CA 90248"
2,2014-12-21,25.94,USD,"Hialeah, FL 33010"
3,2017-05-12,45.58,USD,Ortega Pizzeria & Tavern
4,2016-11-12,117.00,USD,JBR
...,...,...,...,...
195,NaN,24.47,USD,Dona Mercedes Restaurant
196,NaN,9.58,USD,"11732 ARTESIA BLVD. ARTESIA, CA."
197,NaN,40.30,USD,"18128 Pioneer Blvd, Artesia, CA 90701"
198,NaN,10.23,USD,"Golden Coin, 94-450 Ukee St, Waipahu, HI 96797"
